In [4]:
import xarray as xr
import numpy as np
import datetime as dt
import pyfesom2 as pf
import matplotlib.tri as mtri
from joblib import Parallel, delayed
import pickle

In [5]:
datapath='PATH/TO/DATA/'
mesh_path='/PATH/TO/MESH/DATA/'
mesh=pf.load_mesh(mesh_path)

/work/ab0995/a270088/meshes/SO3/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ab0995/a270088/meshes/SO3/pickle_mesh_py3_fesom2


In [6]:
model_lons=mesh.x2
model_lats=mesh.y2

elements=mesh.elem.astype('int32')

d = model_lons[elements].max(axis=1) - model_lons[elements].min(axis=1)
no_cyclic_elem = np.argwhere(d < 100).ravel()

#regular grid for interpolation
dx=0.25
dy=0.25
left,right=-179.875,180
bottom,top=-89.875,90
nx2=left-right
ny2=top-bottom
lon_eq = np.arange(left, right, dx) 
lat_eq = np.arange(bottom, top, dy)
nx=lon_eq.shape[0]
ny=lat_eq.shape[0]

xx_eq, yy_eq = np.meshgrid(lon_eq, lat_eq)
xx_eq=xx_eq.T;
yy_eq=yy_eq.T;

In [7]:
triang = mtri.Triangulation(model_lons, model_lats, elements[no_cyclic_elem])
tri = triang.get_trifinder()

In [3]:
#mean for the entire period
so3_eke15=xr.open_dataset(datapath+'eke_mean_2016-2020.nc')
so3_eke50=xr.open_dataset(datapath+'eke_mean_1951-1955.nc')
so3_eke90=xr.open_dataset(datapath+'eke_mean_2091-2095.nc')

In [5]:
def convert_to_reg(i,var):
    return mtri.LinearTriInterpolator(triang, data[var][i,:].data,trifinder=tri)(xx_eq, yy_eq)

In [10]:
eke_reg50 = mtri.LinearTriInterpolator(triang, so3_eke50.unod[0,5,:].data,trifinder=tri)(xx_eq, yy_eq)
eke_reg15 = mtri.LinearTriInterpolator(triang, so3_eke15.unod[0,5,:].data,trifinder=tri)(xx_eq, yy_eq)
eke_reg90 = mtri.LinearTriInterpolator(triang, so3_eke90.unod[0,5,:].data,trifinder=tri)(xx_eq, yy_eq)

In [11]:
eke_interp50=xr.DataArray(data=eke_reg50.reshape(1,eke_reg50.shape[0],eke_reg50.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':so3_eke50.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2',depth='25m'))

eke_ds50=eke_interp50.to_dataset(name='eke')
eke_ds50.to_netcdf(datapath+'eke_mean_reg_25m_1951-1955.nc')

In [12]:
eke_interp15=xr.DataArray(data=eke_reg15.reshape(1,eke_reg15.shape[0],eke_reg15.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':so3_eke15.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2',depth='25m'))

eke_ds15=eke_interp15.to_dataset(name='eke')
eke_ds15.to_netcdf(datapath+'eke_mean_reg_25m_2016-2020.nc')

In [13]:
eke_interp90=xr.DataArray(data=eke_reg90.reshape(1,eke_reg90.shape[0],eke_reg90.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':so3_eke90.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2',
                               depth='25m'))

eke_ds90=eke_interp90.to_dataset(name='eke')
eke_ds90.to_netcdf(datapath+'eke_mean_reg_25m_2091-2095.nc')

In [1]:
#cmip6 data

In [7]:
gmesh_path='/PATH/TO/MESH/DATA/'
gmesh=pf.load_mesh(gmesh_path)
gmodel_lons=gmesh.x2
gmodel_lats=gmesh.y2
gelements=gmesh.elem.astype('int32')

gd = gmodel_lons[gelements].max(axis=1) - gmodel_lons[gelements].min(axis=1)
gno_cyclic_elem = np.argwhere(gd < 100).ravel()

/pool/data/AWICM/FESOM1/MESHES/glob/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /pool/data/AWICM/FESOM1/MESHES/glob/pickle_mesh_py3_fesom2


In [9]:
gtriang = mtri.Triangulation(gmodel_lons, gmodel_lats, gelements[gno_cyclic_elem])
gtri = gtriang.get_trifinder()

In [ ]:
#ens mean
c6_eke50=xr.open_dataset(datapath+'c6_eke_ensmean_annmean_1951-1955.nc')
c6_eke15=xr.open_dataset(datapath+'c6_eke_ensmean_annmean_2016-2020.nc')
c6_eke90=xr.open_dataset(datapath+'c6_eke_ensmean_annmean_2091-2095.nc')


In [ ]:
geke_reg50 = mtri.LinearTriInterpolator(gtriang, c6_eke50.uo[0,:].data,trifinder=gtri)(xx_eq, yy_eq)
geke_reg15 = mtri.LinearTriInterpolator(gtriang, c6_eke15.uo[0,:].data,trifinder=gtri)(xx_eq, yy_eq)
geke_reg90 = mtri.LinearTriInterpolator(gtriang, c6_eke90.uo[0,:].data,trifinder=gtri)(xx_eq, yy_eq)

In [ ]:
geke_interp50=xr.DataArray(data=geke_reg50.reshape(1,geke_reg50.shape[0],geke_reg50.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':c6_eke50.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2'))
geke_ds50=geke_interp50.to_dataset(name='eke')
geke_ds50.to_netcdf(datapath+'eke_mean_reg_1951-1955.nc')

In [ ]:
geke_interp15=xr.DataArray(data=geke_reg15.reshape(1,geke_reg15.shape[0],geke_reg15.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':c6_eke15.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2'))
geke_ds15=geke_interp15.to_dataset(name='eke')
geke_ds15.to_netcdf(datapath+'eke_mean_reg_2016-2020.nc')

In [ ]:
geke_interp90=xr.DataArray(data=geke_reg90.reshape(1,geke_reg90.shape[0],geke_reg90.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':c6_eke90.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2'))
geke_ds90=geke_interp90.to_dataset(name='eke')
geke_ds90.to_netcdf(datapath+'eke_mean_reg_2091-2095.nc')

In [ ]:
#ens members annual

In [ ]:
for run in [1,2,3,4,5]:
    for time in ['1951-1955','2016-2020','2091-2095']:
        data=xr.open_dataset(datapath+'c6_eke_r'+str(run)+'_annmean_'+time+'.nc')

        geke_reg = mtri.LinearTriInterpolator(gtriang, data.uo[0,:].data,trifinder=gtri)(xx_eq, yy_eq)

        geke_interp=xr.DataArray(data=geke_reg.reshape(1,geke_reg.shape[0],geke_reg.shape[1]),dims=['time','lon','lat'],name='eke',
                        coords={'time':data.time.values.reshape(1),'lon':lon_eq,'lat':lat_eq},
                        attrs=dict(description='eke computed from model output velocity on the SO3 mesh and interpolated to a regular grid',
                               units='m^2/s^2'
                                  ))
        geke_ds=geke_interp.to_dataset(name='eke')
        geke_ds.to_netcdf(datapath+'eke_mean_reg_r'+str(run)+'_'+time+'.nc')